# Debugging autoreload

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages

In [1]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import pathlib

def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Load data

In [3]:
path = f"D:/YandexDisk/Work/bbd/atlas"

## Initial preprocessing

In [ ]:
data_raw = pd.read_excel(f"{path}/Выгрузка биовозраст 2.2.xlsx", sheet_name='Итоговая выгрузка', index_col="№ карты")

# Возраст
age_parts = data_raw['Возраст'].astype(str).str.extract(r"^(\d+)\D*(\d*)\D*$")
age_parts.columns = ['Years', 'Month']
age_parts['Month'] = age_parts['Month'].replace({'': 0})
age_parts['Возраст'] = pd.to_numeric(age_parts['Years']) + pd.to_numeric(age_parts['Month']) / 12.0
data_raw.loc[data_raw.index, 'Возраст'] = age_parts.loc[data_raw.index, 'Возраст']
data_raw['Возраст'] = pd.to_numeric(data_raw['Возраст'])

# Давление
blood_pressure = data_raw['АД'].astype(str).str.extract(r"^(\d+)/(\d+)$")
blood_pressure.columns = ['Cистолическое АД', 'Диастолическое АД']
data_raw.loc[data_raw.index, ['Cистолическое АД', 'Диастолическое АД']] = blood_pressure.loc[data_raw.index, ['Cистолическое АД', 'Диастолическое АД']]
data_raw['Cистолическое АД'] = pd.to_numeric(data_raw['Cистолическое АД'])
data_raw['Диастолическое АД'] = pd.to_numeric(data_raw['Диастолическое АД'])

# % свободного ПСА
data_raw.rename(columns={'% свободного ПСА, %': '% свободного ПСА'}, inplace=True)

# Много знаков %
data_raw.columns = data_raw.columns.str.replace(" %, %", ", %", regex=True)
data_raw.columns = data_raw.columns.str.replace(",,", ",", regex=True)

# Двойные пробелы
data_raw.columns = data_raw.columns.str.replace("  ", " ", regex=True)

# Двойные пробелы
data_raw.columns = data_raw.columns.str.replace("  ", " ", regex=True)

# Запятая c пробелом в конце названия столбца
data_raw.columns = [x[:-2] if x[-2:] == ', ' else x for x in data_raw.columns]

# Пробел перед запятой
data_raw.columns = data_raw.columns.str.replace(" ,", ",", regex=True)

# Replace nan-characters with nan
data_raw.replace({'.': np.nan, }, inplace=True)
data_raw.replace({'-': np.nan, }, inplace=True)
data_raw.replace({r"^.*Необходимо повторить исследование.*$": np.nan}, regex=True, inplace=True)
data_raw.replace({r"^.*расчет невозможен.*$": np.nan}, regex=True, inplace=True)
data_raw.replace({r"^.*нет результата.*$": np.nan}, regex=True, inplace=True)
data_raw.replace({r"^.*без особенностей.*$": np.nan}, regex=True, inplace=True)
data_raw.replace({r'^([<>].*)$': np.nan}, regex=True, inplace=True)
data_raw['Тироксин свободный (Т4 свободный), пмоль/л'] = pd.to_numeric(data_raw['Тироксин свободный (Т4 свободный), пмоль/л'], errors='coerce')

# Уникальность столбцов
cols_dupl = data_raw.columns.values[data_raw.columns.duplicated()]
if len(cols_dupl) > 0:
    print(cols_dupl)

## Save available features for manual selection

In [5]:
df_feats = pd.DataFrame(index=data_raw.columns.values)
df_feats['Selected'] = 1
df_feats.to_excel(f"{path}/feats.xlsx")

In [ ]:
print(data_raw.dtypes[data_raw.dtypes.isin(['object','category'])].index)
data_raw.select_dtypes(include=['object', 'category'])

## Old data: load processed features and apply filters

In [4]:
df_feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
for col in df_feats.index[df_feats['Source'] == 'Inbody'].values:
    data_raw[col] = pd.to_numeric(data_raw[col].astype(str).str.replace(',', '.'), errors='coerce')

## Filter features and save preprocessed data

In [6]:
df_feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
data = data_raw.loc[:, df_feats.index[df_feats['Selected'] == 1].to_list()]
data.to_excel(f"{path}/data.xlsx")

## Load preprocessed data

In [8]:
data = pd.read_excel(f"{path}/data.xlsx", index_col=0)
df_feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)

# NaNs analysis

In [ ]:
nan_pct = data.isna().sum().sum() / data.size * 100
print(nan_pct)

In [ ]:
nan_feats = data.isna().sum(axis=0).to_frame(name="Number of NaNs")
nan_feats["% of NaNs"] = nan_feats["Number of NaNs"] / data.shape[0] * 100
nan_feats["Number of not-NaNs"] = data.notna().sum(axis=0)
nan_feats.sort_values(["% of NaNs"], ascending=[True], inplace=True)
nan_feats.to_excel(f"{path}/nan_feats.xlsx", index_label="Features")

sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(16, 4))
hist = sns.histplot(
    data=nan_feats,
    x="% of NaNs",
    bins=np.linspace(-0.5, 100.5, 102),
    # discrete=True,
    edgecolor='k',
    linewidth=0.5,
    color='crimson',
    ax=ax
)
hist.set(xlim=(-0.5, 100.5))
hist.set_ylabel("Количество признаков")
hist.set_xlabel("% пропущенных значений")
plt.savefig(f"{path}/nan_feats_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/nan_feats_hist.pdf", bbox_inches='tight')
plt.close(fig)

In [10]:
sns.set_theme(style='ticks')
plt.figure(figsize=(4, 15))
df_fig = nan_feats.loc[nan_feats[f"% of NaNs"] < 85, :]
barplot = sns.barplot(
    data=df_fig,
    x=f"% of NaNs",
    y=df_fig.index,
    edgecolor='black',
    dodge=False,
)
for container in barplot.containers:
    barplot.bar_label(container, label_type='edge', fmt='%.2f', fontsize=10, padding=2.5)
barplot.set_xlabel("% пропущенных значений")
plt.savefig(f"{path}/nan_feats_bar.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/nan_feats_bar.pdf", bbox_inches='tight')
plt.close()

# Correlation with Age of all biomarkers

In [ ]:
data = pd.read_excel(f"{path}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
nan_feats =  pd.read_excel(f"{path}/nan_feats.xlsx", index_col=0)
feats = feats.loc[nan_feats.index, :]
feats.loc[feats.index, nan_feats.columns.to_list()] = nan_feats.loc[feats.index, nan_feats.columns.to_list()]
feats_cnt_wo_age = np.sort(feats.index[feats['Type'] == 'continuous'].to_list())
feats_cnt_wo_age = list(feats_cnt_wo_age[feats_cnt_wo_age != 'Возраст'])

hist_bins = np.linspace(5, 115, 23)
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(6, 3.5))
histplot = sns.histplot(
    data=data,
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    x="Возраст",
    color='crimson',
    ax=ax
)
histplot.set(xlim=(0, 120))
plt.savefig(f"{path}/age_hist.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/age_hist.pdf", bbox_inches='tight')
plt.close(fig)

df_corr = pd.DataFrame(index=feats_cnt_wo_age, columns=['count', 'rho', 'pval', ])
for f in tqdm(feats_cnt_wo_age):
    df_tmp = data.loc[:, ['Возраст', f]].dropna(axis=0, how='any')
    if df_tmp.shape[0] > 1:
        df_corr.at[f, 'count'] = df_tmp.shape[0]
        vals_1 = df_tmp.loc[:, 'Возраст'].values
        vals_2 = df_tmp.loc[:, f].values
        df_corr.at[f, 'rho'], df_corr.at[f, 'pval'] = stats.pearsonr(vals_1, vals_2)
df_corr.dropna(axis=0, how='any', inplace=True)
_, df_corr['pval_fdr_bh'], _, _ = multipletests(df_corr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
df_corr.insert(1, "abs(rho)", df_corr['rho'].abs())
df_corr.sort_values(["abs(rho)"], ascending=[False], inplace=True)
df_corr.to_excel(f"{path}/age_pearson.xlsx", index_label="Features")

# Generate datasets

In [ ]:
feats_set = 'lab'

data = pd.read_excel(f"{path}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
nan_feats =  pd.read_excel(f"{path}/feats_{feats_set}.xlsx", index_col=0)
feats = feats.loc[nan_feats.index, :]
feats.loc[feats.index, nan_feats.columns.to_list()] = nan_feats.loc[feats.index, nan_feats.columns.to_list()]
data = data[feats.index.values]
data.dropna(axis=0, how='any', inplace=True)
pathlib.Path(f"{path}/subset_{feats_set}").mkdir(parents=True, exist_ok=True)
feats.to_excel(f"{path}/subset_{feats_set}/feats.xlsx", index_label="Features")
data.to_excel(f"{path}/subset_{feats_set}/data.xlsx", index_label="№ карты")

feats_cnt = np.sort(feats.index[feats['Type'] == 'continuous'].to_list())
feats_cnt = ['Возраст'] + list(feats_cnt[feats_cnt != 'Возраст'])
df_corr = pd.DataFrame(data=np.zeros(shape=(len(feats_cnt), len(feats_cnt))), index=feats_cnt, columns=feats_cnt)
for f_id_1 in range(len(feats_cnt)):
    for f_id_2 in range(f_id_1, len(feats_cnt)):
        f_1 = feats_cnt[f_id_1]
        f_2 = feats_cnt[f_id_2]
        if f_id_1 != f_id_2:
            vals_1 = data.loc[:, f_1].values
            vals_2 = data.loc[:, f_2].values
            corr, pval = stats.pearsonr(vals_1, vals_2)
            df_corr.at[f_2, f_1] = pval
            df_corr.at[f_1, f_2] = corr
        else:
            df_corr.at[f_2, f_1] = np.nan
selection = np.tri(df_corr.shape[0], df_corr.shape[1], -1, dtype=bool)
df_fdr = df_corr.where(selection).stack().reset_index()
df_fdr.columns = ['row', 'col', 'pval']
_, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
df_corr_fdr = df_corr.copy()
for line_id in range(df_fdr.shape[0]):
    df_corr_fdr.loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = -np.log10(df_fdr.at[line_id, 'pval_fdr_bh'])
df_corr_fdr.to_excel(f"{path}/subset_{feats_set}/feats_pearsonr.xlsx")
sns.set_theme(style='ticks')
fig, ax = plt.subplots(figsize=(4.5 + 0.33 * len(feats_cnt), 2.0 + 0.27 * len(feats_cnt)))
cmap_triu = plt.get_cmap("seismic").copy()
mask_triu=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool)
heatmap_diff = sns.heatmap(
    df_corr_fdr,
    mask=mask_triu,
    annot=True,
    fmt=".2f",
    center=0.0,
    cmap=cmap_triu,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 5},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"Pearson $\rho$", size=13)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
cmap_tril = plt.get_cmap("viridis").copy()
cmap_tril.set_under('black')
mask_tril=np.tri(len(feats_cnt), len(feats_cnt), -1, dtype=bool).T
heatmap_pval = sns.heatmap(
    df_corr_fdr,
    mask=mask_tril,
    annot=True,
    fmt=".1f",
    vmin=-np.log10(0.05),
    cmap=cmap_tril,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 5},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('', fontsize=16)
ax.set_ylabel('', fontsize=16)
ax.set_title('', fontsize=16)
# ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
# for tick_label in ax.get_xticklabels():
    # tick_label.set_color(colors_tissues[tick_label.get_text()])
    # ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
# for tick_label in ax.get_yticklabels():
    # tick_label.set_color(colors_tissues[tick_label.get_text()])
    # ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.5, foreground="black")])
# plt.savefig(f"{path}/subset_{feats_set}/feats_pearsonr.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/subset_{feats_set}/feats_pearsonr.pdf", bbox_inches='tight')
plt.close(fig)

# PhenoAge in Atlas

In [ ]:
data = pd.read_excel(f"{path}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
phenoage = pd.read_excel(f"D:/YandexDisk/Work/bbd/phenoage/phenoage.xlsx", index_col=0)
data = data.loc[:, phenoage['Atlas'].values].dropna(axis=0, how='any')
data['NonLog Среактивный белок ультрачувствительный, мг/л'] = data['Среактивный белок ультрачувствительный, мг/л'].values
data['Среактивный белок ультрачувствительный, мг/л'] = np.log(data['Среактивный белок ультрачувствительный, мг/л'].values * 10.0) / np.log(10.0)
data['LinearComb'] = -19.9067
gamma = 0.0077
for f in phenoage.index.values:
    data['LinearComb'] += phenoage.at[f, 'Coeff'] * data[phenoage.at[f, 'Atlas']].values
data['MortalityScore'] = 1 - np.exp(-np.exp(data['LinearComb'].values) * (np.exp(120 * gamma) - 1) / gamma)
data['PhenoAge'] = 141.50225 + np.log(-0.00553 * np.log(1 - data['MortalityScore'].values)) / 0.090165
data['PhenoAge acceleration'] = data['PhenoAge'] - data['Возраст']
print(np.mean(data['PhenoAge acceleration'].values))
data.to_excel(f"{path}/data_PhenoAge.xlsx")

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=False, column_widths=[5, 3], horizontal_spacing=0.15)
min_plot_age = data[["Возраст", "PhenoAge"]].min().min()
max_plot_age = data[["Возраст", "PhenoAge"]].max().max()
shift_plot_age = max_plot_age - min_plot_age
min_plot_age -= 0.1 * shift_plot_age
max_plot_age += 0.1 * shift_plot_age
fig.add_trace(
    go.Scatter(
        x=[min_plot_age, max_plot_age],
        y=[min_plot_age, max_plot_age],
        showlegend=False,
        mode='lines',
        line = dict(color='black', width=2, dash='dot')
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Scatter',
        x=data.loc[:, 'Возраст'].values,
        y=data.loc[:, 'PhenoAge'].values,
        text=data.index.values,
        hovertext=data.index.values,
        showlegend=False,
        mode='markers',
        marker=dict(
            size=10,
            opacity=0.75,
            line=dict(
                width=1,
                color='black'
            ),
            color='crimson'
        )
    ),
    row=1,
    col=1
)
fig.update_xaxes(
    row=1,
    col=1,
    automargin=True,
    title_text="Age",
    autorange=False,
    range=[min_plot_age, max_plot_age],
    showgrid=False,
    zeroline=False,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.update_yaxes(
    row=1,
    col=1,
    automargin=True,
    title_text=f"PhenoAge",
    # scaleanchor="x",
    # scaleratio=1,
    autorange=False,
    range=[min_plot_age, max_plot_age],
    showgrid=False,
    zeroline=False,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.add_trace(
    go.Violin(
        y=data.loc[:, 'PhenoAge acceleration'].values,
        hovertext=data.index.values,
        name="Violin",
        box_visible=True,
        meanline_visible=True,
        showlegend=False,
        line_color='black',
        fillcolor='crimson',
        marker=dict(color='crimson', line=dict(color='black', width=0.5), opacity=0.75),
        points='all',
        bandwidth=np.ptp(data.loc[:, 'PhenoAge acceleration'].values) / 32,
        opacity=0.75
    ),
    row=1,
    col=2
)
fig.update_yaxes(
    row=1,
    col=2,
    automargin=True,
    title_text="PhenoAge acceleraton",
    autorange=True,
    showgrid=False,
    zeroline=True,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.update_xaxes(
    row=1,
    col=2,
    automargin=True,
    autorange=False,
    range=[-0.5, 0.3],
    showgrid=False,
    showline=True,
    zeroline=False,
    showticklabels=False,
    mirror=True,
    ticks='outside',
    tickvals=[],

)
fig.update_layout(
    template="simple_white",
    width=800,
    height=450,
    margin=go.layout.Margin(l=100, r=20, b=50, t=50, pad=0),
)
fig.show()
fig.write_image(f"{path}/PhenoAge.png")
fig.write_image(f"{path}/PhenoAge.pdf", format="pdf")